# Train a Model with SageMaker Autopilot

We will use Autopilot to predict the star rating of customer reviews. Autopilot implements a transparent approach to AutoML. 

For more details on Autopilot, have a look at this [Amazon Science Publication](https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf)

<img src="img/autopilot-transparent.png" width="80%" align="left">

# Introduction

Amazon SageMaker Autopilot is a service to perform automated machine learning (AutoML) on your datasets.  Autopilot is available through the UI or AWS SDK.  In this notebook, we will use the AWS SDK to create and deploy a text processing and star rating classification machine learning pipeline.

# Setup

Let's start by specifying:

* The S3 bucket and prefix to use to train our model.  _Note:  This should be in the same region as this notebook._
* The IAM role of this notebook needs access to your data.

# Notes
* This notebook will take some time to finish. 

* You can start this notebook and continue to the next notebooks whenever you are waiting for the current notebook to finish.

# Checking Pre-Requisites From The Previous `01_Prepare_Dataset_Autopilot` Notebook

In [ ]:
%store -r autopilot_train_s3_uri

In [ ]:
try:
    autopilot_train_s3_uri
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
print(autopilot_train_s3_uri)

In [ ]:
if not autopilot_train_s3_uri:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Training Data

In [ ]:
print(autopilot_train_s3_uri)

In [ ]:
!aws s3 ls $autopilot_train_s3_uri

## See our prepared training data which we use as input for Autopilot

In [ ]:
!aws s3 cp $autopilot_train_s3_uri ./tmp/

In [ ]:
import csv

df = pd.read_csv('./tmp/amazon_reviews_us_Digital_Software_v1_00_autopilot.csv')
df.head()

# Setup the S3 Location for the Autopilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [ ]:
prefix_model_output = 'models/autopilot'

model_output_s3_uri = 's3://{}/{}'.format(bucket, prefix_model_output)

print(model_output_s3_uri)

In [ ]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(autopilot_train_s3_uri)
        }
      },
      'TargetAttributeName': 'star_rating'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Check For Existing Autopilot Jobs

In [ ]:
existing_jobs_response = sm.list_auto_ml_jobs()

In [ ]:
existing_jobs_response

In [ ]:
num_existing_jobs = 0 
running_jobs = 0

if 'AutoMLJobSummaries' in existing_jobs_response.keys():
    job_list = existing_jobs_response['AutoMLJobSummaries']
    num_existing_jobs = len(job_list)
    print('[INFO] You already created {} Autopilot job(s) in this account.'.format(num_existing_jobs))
    for j in job_list:
        if 'AutoMLJobStatus' in j.keys():                
            if j['AutoMLJobStatus'] == 'InProgress':
                running_jobs = running_jobs + 1
    print('[INFO] There are currently {} Autopilot job(s) actively running.'.format(running_jobs))
else:
    print('[OK] Please continue.')

# Launch the SageMaker Autopilot Job

## _Note: Please Only Run This Once._

In [ ]:
from time import gmtime, strftime, sleep

In [ ]:
%store -r auto_ml_job_name

try:
    auto_ml_job_name
except NameError:    
    timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
    auto_ml_job_name = 'automl-dm-' + timestamp_suffix
    print('Created AutoMLJobName: ' + auto_ml_job_name)

In [ ]:
print(auto_ml_job_name)

In [ ]:
%store auto_ml_job_name

In [ ]:
print('Currently Running Jobs (Should be 0): {}'.format(running_jobs))

In [ ]:
max_running_jobs = 1

if running_jobs < max_running_jobs: # Limiting to max. 1 Jobs
    try:
        sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                              InputDataConfig=input_data_config,
                              OutputDataConfig=output_data_config,
                              AutoMLJobConfig=job_config,
                              RoleArn=role)
        print('[OK] Autopilot Job {} created.'.format(auto_ml_job_name))
        running_jobs = running_jobs + 1
    except:
        print('[WARN] You have already launched an Autopilot job.  Please continue see the output of this job.'.format(running_jobs))
else:
    print('[WARN] You have already launched {} Autopilot running job(s).  Please continue see the output of the running job.'.format(running_jobs))

# Track the Progress of the Autopilot Job

SageMaker Autopilot job consists of the following high-level steps: 
* _Data Analysis_ where the data is summarized and analyzed to determine which feature engineering techniques, hyper-parameters, and models to explore.
* _Feature Engineering_ where the data is scrubbed, balanced, combined, and split into train and validation.
* _Model Training and Tuning_ where the top performing features, hyper-parameters, and models are selected and trained.

<img src="img/autopilot-steps.png" width="90%" align="left">

**Autopilot Research Paper: https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf**

# Analyzing Data and Generate Notebooks

In [ ]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while 'AutoMLJobStatus' not in job_description_response.keys() and 'AutoMLJobSecondaryStatus' not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job has not yet started. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for Autopilot Job to start...')
    sleep(15)

print('[OK] AutoMLJob started.')

# Review the SageMaker `Processing Jobs`
* First Processing Job (Data Splitter) checks the data sanity, performs stratified shuffling and splits the data into training and validation. 
* Second Processing Job (Candidate Generator) first streams through the data to compute statistics for the dataset. Then, uses these statistics to identify the problem type, and possible types of every column-predictor: numeric, categorical, natural language, etc.

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">Processing Jobs</a></b>'.format(region)))


# The Next Cell Will Show `InProgress` For A Few Minutes.

## _Please be patient._

In [ ]:
%%time

job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('Starting', 'AnalyzingData'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(15)
    print('[OK] Data analysis phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

# View Generated Notebook Samples
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data Exploration
* Candidate Definition

# Waiting For Generated Notebooks

In [ ]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while 'AutoMLJobArtifacts' not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job has not yet generated the artifacts. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for AutoMLJobArtifacts...')
    sleep(15)

print('[OK] AutoMLJobArtifacts generated.')

In [ ]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while 'DataExplorationNotebookLocation' not in job_description_response['AutoMLJobArtifacts'].keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print('[INFO] Autopilot Job has not yet generated the notebooks. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for DataExplorationNotebookLocation...')
    sleep(15)

print('[OK] DataExplorationNotebookLocation found.')   

In [ ]:
generated_resources = job_description_response['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')

pr_job_id = generated_resources.rsplit('/', 1)[-1]

In [ ]:
from IPython.core.display import display, HTML

if not pr_job_id: 
    print('No AutoMLJobArtifacts found.')
else: 
    display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/{}/sagemaker-automl-candidates/{}/">S3 Generated Resources</a></b>'.format(bucket, prefix_model_output, auto_ml_job_name, pr_job_id)))

# In the Jupyter File Browser, Open the Following Folders to See Samples of the Generated Assets:
```
notebooks/
generated_module/
```

Lots of useful information ^^ in these folders ^^

#### _(Optional) You can download the actual files generated for your specific Autopilot run using the following:_
```
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')

!aws s3 cp --recursive $generated_resources .
```

# Feature Engineering

### Watch out for SageMaker `Training Jobs` and `Batch Transform Jobs` to start. 

* This is the candidate exploration phase. 
* Each python script code for data-processing is executed inside a SageMaker framework container as a training job, followed by transform job.

Note, that feature preprocessing part of each pipeline has all hyper parameters fixed, i.e. does not require tuning, thus feature preprocessing step can be done prior runing the hyper parameter optimization job. 

It outputs up to 10 variants of transformed data, therefore algorithms for each pipeline are set to use
the respective transformed data.

<img src="img/autopilot-steps.png" width="90%" align="left">

**Autopilot Research Paper: https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf**

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(region)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/">Batch Transform Jobs</a></b>'.format(region)))


# The Next Cell Will Show `InProgress` For A Few Minutes.

## _Please be patient._ ##

In [ ]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(15)
    print('[OK] Feature engineering phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

# Model Training and Tuning

### Watch out for a SageMaker`Hyperparameter Tuning Job` and various `Training Jobs` to start. 

* All algorithms are optimized using a SageMaker Hyperparameter Tuning job. 
* Up to 250 training jobs (based on number of candidates specified) are selectively executed to find the best candidate model.

<img src="img/autopilot-steps.png" width="90%" align="left">

**Autopilot Research Paper: https://assets.amazon.science/e8/8b/2366b1ab407990dec96e55ee5664/amazon-sagemaker-autopilot-a-white-box-automl-solution-at-scale.pdf**

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/">Hyperparameter Tuning Jobs</a></b>'.format(region)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(region)))


# The Next Cell Will Show `InProgress` For A Few Minutes.

## _Please be patient._

In [ ]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(15)
    print('[OK] Model tuning phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

# _Please Wait Until ^^ Autopilot ^^ Completes Above_

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

Make sure the status below indicates `Completed`.

In [ ]:
if job_status not in ('Completed'):
    print('************************************************************')
    print('[ERROR] THIS JOB DID NOT COMPLETE PROPERLY. ****************')
    print('[ERROR] LOOK IN PREVIOUS CELLS TO FIND THE ISSUE. **********')    
    print('************************************************************')

# Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [ ]:
candidates_response = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                         SortBy='FinalObjectiveMetricValue')

### Check that candidates exist

In [ ]:
if not candidates_response:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
else:
    candidates = candidates_response['Candidates']
    print('[OK]')

In [ ]:
if not candidates:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
elif 'CandidateName' not in candidates[0]:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
elif 'FinalAutoMLJobObjectiveMetric' not in candidates[0]:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
else:
    print('[OK]')

In [ ]:
print(json.dumps(candidates, indent=4, sort_keys=True, default=str))

In [ ]:
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

# Inspect Trials using Experiments API

SageMaker Autopilot automatically creates a new experiment, and pushes information for each trial. 

In [ ]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
print(df)

# Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [ ]:
best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [ ]:
if not best_candidate_response:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
else:
    best_candidate = best_candidate_response['BestCandidate']
    print('[OK]')

In [ ]:
print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))

In [ ]:
if not best_candidate:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
elif 'CandidateName' not in best_candidate:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
elif 'FinalAutoMLJobObjectiveMetric' not in best_candidate:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
else:
    best_candidate_identifier = best_candidate['CandidateName']
    print("Candidate name: " + best_candidate_identifier)
    print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
    print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))


In [ ]:
print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

# View Individual Autopilot Jobs

In [ ]:
steps = []
if not best_candidate:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
elif 'InferenceContainers' not in best_candidate:
    print('[ERROR] THE JOB DID NOT COMPLETE PROPERLY. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
else:
    for step in best_candidate['CandidateSteps']:
        print('Candidate Step Type: {}'.format(step['CandidateStepType']))
        print('Candidate Step Name: {}'.format(step['CandidateStepName']))
        steps.append(step['CandidateStepName'])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, steps[0])))

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a></b>'.format(region, steps[1])))

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}">Transform Job</a></b>'.format(region, steps[2])))

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job (Tuning)</a></b>'.format(region, steps[3])))

# See the containers and models composing the Inference Pipeline

In [ ]:
best_candidate_containers = best_candidate['InferenceContainers']

In [ ]:
for container in best_candidate_containers:
        print(container['Image'])
        print(container['ModelDataUrl'])
        print('======================')

# Update Containers To Show Predicted Label and Confidence Score

In [ ]:
for container in best_candidate_containers:
        print(container['Environment'])
        print('======================')

In [ ]:
best_candidate_containers[1]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability'})
best_candidate_containers[2]['Environment'].update({'SAGEMAKER_INFERENCE_INPUT': 'predicted_label, probability'})
best_candidate_containers[2]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability'})

In [ ]:
for container in best_candidate_containers:
        print(container['Environment'])
        print('======================')

# Autopilot Chooses XGBoost as Best Candidate!

Note that Autopilot chose different hyper-parameters and feature transformations than we used in our own XGBoost model.

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [ ]:
print(best_candidate['InferenceContainers'])

In [ ]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

# Define EndpointConfig Name

In [ ]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

print(epc_name)

# Define REST Endpoint Name for Autopilot Model

In [ ]:
autopilot_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(autopilot_endpoint_name)
print(variant_name)

In [ ]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m5.large',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])

In [ ]:
create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, autopilot_endpoint_name)))

# Store Variables for the Next Notebooks

In [ ]:
%store autopilot_endpoint_name

In [ ]:
%store

# Summary
We used Autopilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

Autopilot uses a transparent approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the Autopilot service.

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();